In [1]:
import sys
import numpy as np
import cv2
import sklearn.metrics as sklm

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

# from keras import backend as K
# img_dim_ordering = 'tf'
# K.set_image_dim_ordering(img_dim_ordering)


### 1. 데이터 불러오기

In [2]:
import cv2

In [3]:
import os

TRAIN_DIR = '../images/mnist/trainingSet/'
train_folder_list = os.listdir(TRAIN_DIR)
train_folder_list

IMG_SIZE = 32
train_images=[]
train_labels=[]
# 폴더를 하나씩 탐색한다!
for index in range(0, len(train_folder_list)):
    path = os.path.join(TRAIN_DIR, train_folder_list[index])
    path = path + '/'
    img_list = os.listdir(path)
    # 이미지를 하나씩 탐색한다!
    for img in img_list:
        img_path = os.path.join(path, img)
        try:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            train_images.append(new_img)
            train_labels.append(index)
        except Exception as e:
            print(e)
            pass

OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



In [4]:
import numpy as np

In [5]:
x_train = np.array(train_images)
y_train = np.array(train_labels)

In [6]:
# # loading the data
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 데이터 사이즈 조정

In [7]:
# # converting it to RGB
# x_train = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_train]
# x_train = np.concatenate([arr[np.newaxis] for arr in x_train]).astype('float32')

In [8]:
# x_test = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_test]
# x_test = np.concatenate([arr[np.newaxis] for arr in x_test]).astype('float32')

In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
y_train_one = to_categorical(y_train)

### 3. 모델 생성

In [11]:
img_shape = x_train.shape[1:]
img_shape

(32, 32, 3)

In [12]:
num_classes = len(set(y_train))
num_classes

10

In [13]:
layer_type="relu"
layer_type

'relu'

In [14]:
# model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv = VGG16(weights=None, include_top=False)
#model_vgg16_conv.summary()

# top layer 수정
keras_input = Input(shape=img_shape, name = 'custom_input')

# 커스탐모델 생성 
output_vgg16_conv = model_vgg16_conv(keras_input)
output_vgg16_conv

<tf.Tensor 'vgg16/Identity:0' shape=(None, 1, 1, 512) dtype=float32>

In [15]:
# Fully Connected Layer 추가 생성 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(64, activation=layer_type, name='custom_1')(x)
x = Dense(64, activation=layer_type, name='custom_2')(x)
x = Dense(num_classes, activation='softmax', name='prediction')(x)
# 커스텀 모델 생성
pretrained_model = Model(inputs=keras_input, outputs=x)

In [16]:
pretrained_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
custom_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
custom_1 (Dense)             (None, 64)                32832     
_________________________________________________________________
custom_2 (Dense)             (None, 64)                4160      
_________________________________________________________________
prediction (Dense)           (None, 10)                650       
Total params: 14,752,330
Trainable params: 14,752,330
Non-trainable params: 0
_________________________________________________

### 4. 모델 컴파일

In [17]:
pretrained_model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

### 5. 모델 훈련

In [ ]:
pretrained_model.fit(x_train, 
                     y_train_one, 
                     epochs = 2, 
                     verbose=1)

Train on 42000 samples
Epoch 1/2
 1856/42000 [>.............................] - ETA: 36:43 - loss: 2.7756 - accuracy: 0.1061